In [33]:
import cv2
import numpy as np
import os

# Caminho para os arquivos
coco_names_path = 'yolo/coco.names'
yolov3_weights_path = 'yolo/yolov3.weights'
yolov3_cfg_path = 'yolo/yolov3.cfg'

# Carregar os nomes das classes
with open(coco_names_path, 'r') as f:
    classes = f.read().strip().split('\n')

# Carregar a rede YOLO
net = cv2.dnn.readNet(yolov3_weights_path, yolov3_cfg_path)

# Capturar vídeo da câmera
cap = cv2.VideoCapture(0)

# Verificar se a câmera foi aberta corretamente
if not cap.isOpened():
    print("Erro ao abrir a câmera")
    exit()

frame_count = 0
output_dir = 'output_frames'
os.makedirs(output_dir, exist_ok=True)

while True:
    # Ler um frame do vídeo
    ret, frame = cap.read()
    
    # Verificar se o frame foi capturado corretamente
    if not ret:
        print("Falha ao capturar o frame da câmera")
        break
    
    # Obter as dimensões do frame
    height, width = frame.shape[:2]
    
    # Criar um blob a partir do frame
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    # Obter as saídas da rede
    layer_names = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()
    
    # Verificar o formato dos índices retornados
    if isinstance(unconnected_out_layers, np.ndarray):
        unconnected_out_layers = unconnected_out_layers.flatten()
    
    try:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers]
    except IndexError as e:
        print(f"Erro ao acessar os índices das camadas: {e}")
        break
    
    detections = net.forward(output_layers)
    
    # Processar as detecções
    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                # Obter as coordenadas da caixa delimitadora
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                # Desenhar a caixa delimitadora e o rótulo
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                label = f"{classes[class_id]}: {confidence:.2f}"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Salvar o frame com as detecções
    frame_filename = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_filename, frame)
    frame_count += 1
    
    # Sair do loop ao pressionar a tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar a captura
cap.release()


Falha ao capturar o frame da câmera
